# Welcome

## IMPORT REQUIRED DEPENDENCIES

In [2]:
import os
from datetime import datetime

import tensorflow as tf

import experiment_runner.models_config as models
import experiment_runner.datasets_config as datasets
import experiment_runner.feature_extractors_config as feature_extractors

In [3]:
from absl import logging
logging.set_verbosity(logging.ERROR)

## Set Hyper Parameters

### EXPERIMENT PARAMETERS

In [11]:
EXPERIMENT_NAME = 'Movenet_test'
EXPERIMENT_PARAMS = [
    {'name': 'gru',
     'batch_size': 64,
     'epochs': 2,
     },
]

### DATA PARAMETERS

In [12]:
DATASETS_PARAMS = [
    {'dataset_path': './datasets/UCF-3',
     'seq_len': 50,
     'train_test_split': .8
     },
]

### EXTRACTOR PARAMETERS

In [13]:
EXTRACTOR_PARAMS = [
    (feature_extractors.MovenetExtractor, {}),
]

### MODEL PARAMETERS

In [14]:
MODEL_PARAMS = [
    (models.GRU.gru2, {'activation_function': 'relu',
                       'loss_function': 'sparse_categorical_crossentropy',
                       'optimizer': 'adam',
                       }),
]

 ## Create train function

In [15]:
# TRAIN MODEL AND SAVE TO saved_experiments DIRECTORY
def train_model(model, dataset, experiment_params):
    experiment_dir = os.path.join(f'../saved_experiments/{EXPERIMENT_NAME}')

    log_dir = os.path.join(experiment_dir, 'logs/fit/',
                           experiment_params['name'] + '_' + datetime.now().strftime("%Y%m%d-%H%M%S"))

    save_model_callback = tf.keras.callbacks.ModelCheckpoint(os.path.join(experiment_dir, experiment_params['name']),
                                                             monitor='val_loss',
                                                             verbose=1,
                                                             save_best_only=True,
                                                             options=None,)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                          histogram_freq=1,
                                                          update_freq='epoch',)

    out = model.fit(
        dataset.train_dataset,
        validation_data=dataset.validation_dataset,
        epochs=experiment_params['epochs'],
        batch_size=experiment_params['batch_size'],
        callbacks=[tensorboard_callback, save_model_callback],
    )
    
    # evaluate
    print('==== EVAL ====')
    res = model.evaluate(
        dataset.test_dataset
    )
    
    test_summary_writer = tf.summary.create_file_writer(os.path.join(log_dir, 'eval'))
    with test_summary_writer.as_default():
        tf.summary.scalar('test_loss', res[0], step=0)
        tf.summary.scalar('test_accuracy', res[1], step=0)
        
    return out

In [18]:
from tensorboard import notebook
log_dir = os.path.join(f'../saved_experiments/{EXPERIMENT_NAME}/logs')
%load_ext tensorboard
%tensorboard --logdir {log_dir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 6392), started 0:00:47 ago. (Use '!kill 6392' to kill it.)

In [17]:
if __name__ == '__main__':
    # train test loop
    for idx, (experiment_params, data_params, extractor_params, model_params) in \
            enumerate(zip(EXPERIMENT_PARAMS, DATASETS_PARAMS, EXTRACTOR_PARAMS, MODEL_PARAMS)):

        # init based on hyper parameters
        extractor = extractor_params[0](**extractor_params[1])  # get extractor
        dataset = datasets.Dataset.Training(**data_params, extractor=extractor)  # get data
        model = model_params[0](output_size=len(dataset.labels), **model_params[1])  # get model

        train_model(model, dataset, experiment_params)  # train model
        # test_model(model, dataset, experiment_params)  # evaluate model

Epoch 1/2
    179/Unknown - 9s 28ms/step - loss: 0.5538 - sparse_categorical_accuracy: 0.7450
Epoch 1: val_loss improved from inf to 0.51767, saving model to ../saved_experiments/Movenet_test_1\gru
INFO:tensorflow:Assets written to: ../saved_experiments/Movenet_test_1\gru\assets


INFO:tensorflow:Assets written to: ../saved_experiments/Movenet_test_1\gru\assets


180/180 [==============================] - 20s 87ms/step - loss: 0.5528 - sparse_categorical_accuracy: 0.7455 - val_loss: 0.5177 - val_sparse_categorical_accuracy: 0.8629
Epoch 2/2
170/180 [===========================>..] - ETA: 0s - loss: 0.1561 - sparse_categorical_accuracy: 0.9460WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 360 batches). You may need to use the repeat() function when building your dataset.



Epoch 2: val_loss improved from 0.51767 to 0.06923, saving model to ../saved_experiments/Movenet_test_1\gru
INFO:tensorflow:Assets written to: ../saved_experiments/Movenet_test_1\gru\assets


INFO:tensorflow:Assets written to: ../saved_experiments/Movenet_test_1\gru\assets


180/180 [==============================] - 16s 84ms/step - loss: 0.1561 - sparse_categorical_accuracy: 0.9460 - val_loss: 0.0692 - val_sparse_categorical_accuracy: 0.9779
==== EVAL ====
47/47 [==============================] - 1s 14ms/step - loss: 0.0994 - sparse_categorical_accuracy: 0.9535
